In [7]:
import json

In [8]:


class KeyValueStore:
    def __init__(self):
        self.data = {}

    def put(self, key, value):
        keys = key.split('.')
        current_level = self.data
        for k in keys[:-1]:
            if k not in current_level:
                current_level[k] = {}
            current_level = current_level[k]
        current_level[keys[-1]] = value

    def __getitem__(self, key):
        if not isinstance(key, str):
            key = str(key)
        keys = key.split('.')
        current_level = self.data
        for k in keys:
            if k not in current_level:
                return None
            current_level = current_level[k]
        return current_level

    def pop(self, key):
      keys = key.split('.')
      current_level = self.data
      for k in keys[:-1]:
        if k not in current_level:
          return "Key not found"
        current_level = current_level[k]
      if keys[-1] in current_level:
        del current_level[keys[-1]]
        return "Key deleted"
      else:
        return "Key not found"


    def bput(self, python_dict):
        for key, value in python_dict.items():
            self.put(key, value)

    def bget(self, python_dict):
        result = {}
        for key in python_dict:
            result[key] = self[key]
        return result

    def dump(self):
        return self.data

    def __len__(self):
        return len(self.data)



In [9]:

daos_dict = KeyValueStore()

Load data from JSON file and store in daos_dict

In [10]:
with open('data.json') as json_file:
    data = json.load(json_file)
    for entry in data:
        key = entry["usn"]
        value = entry["name"]
        daos_dict.put(key, value)

In [ ]:

while True:
    cmd = input("\nCommand (? for help)> ")
    if cmd == "?":
        print("?\t- Print this help")
        print("r\t- Read a key")
        print("ra\t- Read all keys")
        print("i\t- Insert new key")
        print("d\t- Delete key")
        print("ib\t- Insert new keys in bulk")
        print("rb\t- Read keys in bulk")
        print("q\t- Quit")
    elif cmd == "r":
        key = input("USN? ")
        try:
            print("\tUSN:", key, "\tName:", daos_dict[key])
        except KeyError:
            print("\tError! Key not found")
    elif cmd == "ra":
        print("\nDictionary length =", len(daos_dict))
        for USN, name in daos_dict.dump().items():
            print("USN =", USN, "  Name =", name)
    elif cmd == "i":
        key = input("Enter USN: ")
        value = input("Enter Name: ")
        daos_dict.put(key, value)
        print("Key-value pair inserted successfully.")
    elif cmd == "d":
        key = input("USN to delete? ")
        print(daos_dict.pop(key))
    elif cmd == "ib":
        print("\nInserting new keys in bulk")
        bulk_data = {}
        while True:
            key = input("Enter USN (press Enter to finish): ")
            if key == "":
                break
            value = input("Enter Name for " + key + ": ")
            bulk_data[key] = value
        daos_dict.bput(bulk_data)
        print("Bulk insertion completed.")
    elif cmd == "rb":
        print("\nReading keys in bulk")
        bulk_keys = []
        while True:
            key = input("Enter key to read (press Enter to finish): ")
            if key == "":
                break
            bulk_keys.append(key)
        result = daos_dict.bget({key: None for key in bulk_keys})
        print("Bulk read result:")
        for key, value in result.items():
            print("\tUSN:", key, "\tName:", value)
    elif cmd == "q":
        break
    else:
        print("Invalid command. Please try again.")



Command (? for help)> ?
?	- Print this help
r	- Read a key
ra	- Read all keys
i	- Insert new key
d	- Delete key
ib	- Insert new keys in bulk
rb	- Read keys in bulk
q	- Quit
